In [44]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers , activations , models , preprocessing , utils
import pandas as pd
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
# print( tf.VERSION )


## Reading the data

In [58]:
data_path = 'q3.txt'

In [59]:
input_texts = []
target_texts = []
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(2000, len(lines) - 1)]:
    input_text = line.split('\t')[0]
    target_text = line.split('\t')[1]
    input_texts.append(input_text)
    target_texts.append(target_text)

In [60]:
len(input_texts)

2000

In [61]:
zippedList =  list(zip(input_texts, target_texts))
lines = pd.DataFrame(zippedList, columns = ['input' , 'output']) 

In [62]:
lines.head()

,input,output
0,The kitchen stinks .,I'll throw out the garbage .
1,Getting worse . Now he ’ s eating me out of h...,"Leo , I really think you ’ re beating around ..."
2,"Leo , I really think you ’ re beating around ...",You ’ re right . Everything is probably going...
3,I'm not sure . But I'll get a table ready as ...,OK . We'll wait .
4,Thanks a lot . That's the favor I was going t...,The pleasure is mine .


## Preparing input data for the Encoder

In [63]:
input_lines = list()
for line in lines.input:
    input_lines.append( line ) 

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( input_lines ) 
tokenized_input_lines = tokenizer.texts_to_sequences( input_lines ) 

length_list = list()
for token_seq in tokenized_input_lines:
    length_list.append( len( token_seq ))
max_input_length = np.array( length_list ).max()
print( 'Input max length is {}'.format( max_input_length ))

padded_input_lines = preprocessing.sequence.pad_sequences( tokenized_input_lines , maxlen=max_input_length , padding='post' )
encoder_input_data = np.array( padded_input_lines )
print( 'Encoder input data shape -> {}'.format( encoder_input_data.shape ))

input_word_dict = tokenizer.word_index
num_input_tokens = len( input_word_dict )+1
print( 'Number of Input tokens = {}'.format( num_input_tokens))


Input max length is 148
Encoder input data shape -> (2000, 148)
Number of Input tokens = 2935


## Preparing input data for the Decoder

In [64]:
output_lines = list()
for line in lines.output:
    output_lines.append( '<START> ' + line + ' <END>' )  

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( output_lines ) 
tokenized_output_lines = tokenizer.texts_to_sequences( output_lines ) 

length_list = list()
for token_seq in tokenized_output_lines:
    length_list.append( len( token_seq ))
max_output_length = np.array( length_list ).max()
print( 'Output max length is {}'.format( max_output_length ))

padded_output_lines = preprocessing.sequence.pad_sequences( tokenized_output_lines , maxlen=max_output_length, padding='post' )
decoder_input_data = np.array( padded_output_lines )
print( 'Decoder input data shape -> {}'.format( decoder_input_data.shape ))

output_word_dict = tokenizer.word_index
num_output_tokens = len( output_word_dict )+1
print( 'Number of Output tokens = {}'.format( num_output_tokens))


Output max length is 150
Decoder input data shape -> (2000, 150)
Number of Output tokens = 2561


## Preparing target data for the Decoder 

In [65]:
decoder_target_data = list()
for token_seq in tokenized_output_lines:
    decoder_target_data.append( token_seq[ 1 : ] ) 
    
padded_output_lines = preprocessing.sequence.pad_sequences( decoder_target_data , maxlen=max_output_length, padding='post' )
onehot_output_lines = utils.to_categorical( padded_output_lines , num_output_tokens )
decoder_target_data = np.array( onehot_output_lines )
print( 'Decoder target data shape -> {}'.format( decoder_target_data.shape ))

Decoder target data shape -> (2000, 150, 2561)


## Defining the Model

In [66]:
encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( num_input_tokens, 256 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 256 , return_state=True , recurrent_dropout=0.2 , dropout=0.2 )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( num_output_tokens, 256 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 256 , return_state=True , return_sequences=True , recurrent_dropout=0.2 , dropout=0.2)
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( num_output_tokens , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy')

model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, None, 256)    751360      input_11[0][0]                   
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, None, 256)    655616      input_12[0][0]                   
____________________________________________________________________________________________

## Training

In [67]:
model.fit([encoder_input_data , decoder_input_data], decoder_target_data, batch_size=124, epochs=250) 
model.save( 'model_q3.h5' ) 

Train on 2000 samples
Epoch 1/250
2000/2000 [==============================] - 13s 6ms/sample - loss: 0.6038
Epoch 2/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.4655
Epoch 3/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.4395
Epoch 4/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.4301
Epoch 5/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.4202
Epoch 6/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.4100
Epoch 7/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.4030
Epoch 8/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.3972
Epoch 9/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.3924
Epoch 10/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.3882
Epoch 11/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.3835
Epoch 12/250
2000/2000 [=====

2000/2000 [==============================] - 9s 4ms/sample - loss: 0.1562
Epoch 96/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.1537
Epoch 97/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.1522
Epoch 98/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.1500
Epoch 99/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.1482
Epoch 100/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.1461
Epoch 101/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.1441
Epoch 102/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.1420
Epoch 103/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.1402
Epoch 104/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.1378
Epoch 105/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.1364
Epoch 106/250
2000/2000 [=========================

2000/2000 [==============================] - 9s 5ms/sample - loss: 0.0425
Epoch 189/250
2000/2000 [==============================] - 9s 5ms/sample - loss: 0.0416
Epoch 190/250
2000/2000 [==============================] - 9s 5ms/sample - loss: 0.0409
Epoch 191/250
2000/2000 [==============================] - 9s 5ms/sample - loss: 0.0407
Epoch 192/250
2000/2000 [==============================] - 9s 5ms/sample - loss: 0.0400
Epoch 193/250
2000/2000 [==============================] - 9s 5ms/sample - loss: 0.0397
Epoch 194/250
2000/2000 [==============================] - 9s 5ms/sample - loss: 0.0388
Epoch 195/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.0385
Epoch 196/250
2000/2000 [==============================] - 9s 5ms/sample - loss: 0.0381
Epoch 197/250
2000/2000 [==============================] - 9s 4ms/sample - loss: 0.0374
Epoch 198/250
2000/2000 [==============================] - 9s 5ms/sample - loss: 0.0370
Epoch 199/250
2000/2000 [=====================

## Inference models

In [68]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=(256,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(256,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model


In [69]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( input_word_dict[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_input_length , padding='post')


In [70]:
enc_model , dec_model = make_inference_models()

enc_model.save( 'enc_model_q3.h5' ) 
dec_model.save( 'dec_model_q3.h5' ) 


for epoch in range( encoder_input_data.shape[0] ):
    states_values = enc_model.predict( str_to_tokens( input( 'User: ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = output_word_dict['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in output_word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > max_output_length:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( "Bot:" +decoded_translation.replace(' end', '') )
    print()
    


User: hi
Bot: even him that is your change

User: what do you mean
Bot: thank you i hope so

User: hello
Bot: hello i'd like to have a vacant apartment



KeyboardInterrupt: Interrupted by user